In [18]:

import heapq

def allocate_files(files, num_folders=10):
    total_size = sum(files)
    folder_capacity = 5192
    folders = [{'remaining': folder_capacity, 'index': i} for i in range(num_folders)]
    heap = [(-folder_capacity, i) for i in range(num_folders)]
    heapq.heapify(heap)
    
    allocations = {}
    file_details = []
    
    for idx, size in enumerate(sorted(files, reverse=True)):
        file_details.append((idx, size))
    
    for file_id, size in file_details:
        temp_heap = [(-f['remaining'], f['index']) for f in folders]
        heapq.heapify(temp_heap)
        selected = []
        used = set()
        
        while temp_heap and len(selected) < 3:
            rem_neg, idx = heapq.heappop(temp_heap)
            rem = -rem_neg
            if rem >= size and idx not in used:
                selected.append(idx)
                used.add(idx)
        
        if len(selected) >= 3:
            for idx in selected:
                folders[idx]['remaining'] -= size
            allocations[file_id] = {'split': False, 'locations': selected}
            continue
        
        s1 = size / 2
        s2 = size - s1
        if s1 > folder_capacity or s2 > folder_capacity:
            return None
        
        temp_heap_s1 = [(-f['remaining'], f['index']) for f in folders]
        heapq.heapify(temp_heap_s1)
        s1_selected = []
        used_s1 = set()
        
        while temp_heap_s1 and len(s1_selected) < 3:
            rem_neg, idx = heapq.heappop(temp_heap_s1)
            rem = -rem_neg
            if rem >= s1 and idx not in used_s1:
                s1_selected.append(idx)
                used_s1.add(idx)
        
        if len(s1_selected) < 3:
            return None
        
        for idx in s1_selected:
            folders[idx]['remaining'] -= s1
        
        temp_heap_s2 = [(-f['remaining'], f['index']) for f in folders]
        heapq.heapify(temp_heap_s2)
        s2_selected = []
        used_s2 = set()
        
        while temp_heap_s2 and len(s2_selected) < 3:
            rem_neg, idx = heapq.heappop(temp_heap_s2)
            rem = -rem_neg
            if rem >= s2 and idx not in used_s2:
                s2_selected.append(idx)
                used_s2.add(idx)
        
        if len(s2_selected) < 3:
            return None
        
        for idx in s2_selected:
            folders[idx]['remaining'] -= s2
        
        s1_locations = sorted(s1_selected)
        s2_locations = sorted(s2_selected)
        min_diff = float('inf')
        best_s2 = s2_locations
        
        for i in range(len(s2_locations) - 2):
            current = s2_locations[i:i+3]
            diff = sum(abs(current[j] - s1_locations[j]) for j in range(3))
            if diff < min_diff:
                min_diff = diff
                best_s2 = current
        
        allocations[file_id] = {
            'split': True,
            's1': {'size': s1, 'locations': s1_locations},
            's2': {'size': s2, 'locations': best_s2}
        }
    
    for f in folders:
        if f['remaining'] < 0:
            return None
    
    return allocations

def main():
    files = [1067, 799, 1585, 836, 944, 502, 1399, 783, 494, 712, 631, 1138, 434, 1107, 545, 1489] 
    files = [0.81 * _ for _ in files]
    result = allocate_files(files)
    print (result)
    
    if result is None:
        print("无法找到可行的分配策略。")
        return
    
    for file_id, details in result.items():
        if details['split']:
            print(f"文件 {file_id} 被拆分为两个部分:")
            print(f"  部分1大小 {details['s1']['size']}, 备份位置: {details['s1']['locations']}")
            print(f"  部分2大小 {details['s2']['size']}, 备份位置: {details['s2']['locations']}")
        else:
            print(f"文件 {file_id} 完整存储，备份位置: {details['locations']}")

if __name__ == "__main__":
    main()

{0: {'split': False, 'locations': [0, 1, 2]}, 1: {'split': False, 'locations': [3, 4, 5]}, 2: {'split': False, 'locations': [6, 7, 8]}, 3: {'split': False, 'locations': [9, 6, 7]}, 4: {'split': False, 'locations': [9, 8, 3]}, 5: {'split': False, 'locations': [4, 5, 0]}, 6: {'split': False, 'locations': [1, 2, 9]}, 7: {'split': False, 'locations': [8, 1, 2]}, 8: {'split': False, 'locations': [6, 7, 4]}, 9: {'split': False, 'locations': [5, 3, 0]}, 10: {'split': False, 'locations': [9, 6, 7]}, 11: {'split': False, 'locations': [5, 8, 4]}, 12: {'split': False, 'locations': [1, 2, 3]}, 13: {'split': False, 'locations': [0, 9, 1]}, 14: {'split': False, 'locations': [2, 3, 0]}, 15: {'split': False, 'locations': [5, 8, 4]}}
文件 0 完整存储，备份位置: [0, 1, 2]
文件 1 完整存储，备份位置: [3, 4, 5]
文件 2 完整存储，备份位置: [6, 7, 8]
文件 3 完整存储，备份位置: [9, 6, 7]
文件 4 完整存储，备份位置: [9, 8, 3]
文件 5 完整存储，备份位置: [4, 5, 0]
文件 6 完整存储，备份位置: [1, 2, 9]
文件 7 完整存储，备份位置: [8, 1, 2]
文件 8 完整存储，备份位置: [6, 7, 4]
文件 9 完整存储，备份位置: [5, 3, 0]
文件 10 完整存储，备份

In [20]:
sum([1067, 799, 1585, 836, 944, 502, 1399, 783, 494, 712, 631, 1138, 434, 1107, 545, 1489]) * 3

43395

In [2]:
sorted([[1, 898, 837], [3, 384, 999], [2, 998, 1111]])

[[1, 898, 837], [2, 998, 1111], [3, 384, 999]]

In [1]:
e = [[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [[[4, 11, 4], [4, 11, 5], [4, 11, 6]]], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [[[1, 858, 10], [1, 1607, 9], [1, 821, 4]]], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [[[3, 928, 5], [3, 928, 6], [3, 1196, 1]]], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [[[1, 2680, 9], [1, 2617, 2], [1, 2617, 3]]], [], [], [], [[[5, 2641, 9], [5, 2578, 2], [5, 2578, 3]]], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [[[1, 2464, 7], [1, 2464, 8], [1, 1344, 5]], [[1, 2476, 7], [1, 2476, 8], [1, 1356, 5]]], [[[2, 2477, 7], [2, 2477, 8], [2, 1357, 5]]], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [[[1, 3253, 2], [1, 3253, 3], [1, 2459, 4]]], [[[2, 3254, 2], [2, 3254, 3], [2, 2460, 4]], [[2, 3277, 2], [2, 3277, 3], [2, 2483, 4]]], [[[3, 3328, 2], [3, 3328, 3], [3, 2534, 4]]], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [[[1, 2308, 1], [1, 3837, 10], [1, 3710, 2]]], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [], [[[2, 3689, 3], [2, 2895, 4], [2, 3394, 1]]], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]] 

In [3]:
len(e)

17

In [10]:
len(e[0])


100

In [8]:
cleared_list = []

for _ in e:
    tmp = []
    for l in _:
        if l:
            tmp.append(l)
    cleared_list.append(tmp)

for i, l in enumerate(cleared_list):
    print("tag", i, ":",)
    for j in range(len(cleared_list[i])):
        print(f"len: {j+1}: {cleared_list[i][j]}")

tag 0 :
tag 1 :
tag 2 :
len: 1: [[[4, 11, 4], [4, 11, 5], [4, 11, 6]]]
tag 3 :
tag 4 :
tag 5 :
len: 1: [[[1, 858, 10], [1, 1607, 9], [1, 821, 4]]]
tag 6 :
len: 1: [[[3, 928, 5], [3, 928, 6], [3, 1196, 1]]]
tag 7 :
tag 8 :
len: 1: [[[1, 2680, 9], [1, 2617, 2], [1, 2617, 3]]]
len: 2: [[[5, 2641, 9], [5, 2578, 2], [5, 2578, 3]]]
tag 9 :
len: 1: [[[1, 2464, 7], [1, 2464, 8], [1, 1344, 5]], [[1, 2476, 7], [1, 2476, 8], [1, 1356, 5]]]
len: 2: [[[2, 2477, 7], [2, 2477, 8], [2, 1357, 5]]]
tag 10 :
tag 11 :
tag 12 :
tag 13 :
len: 1: [[[1, 3253, 2], [1, 3253, 3], [1, 2459, 4]]]
len: 2: [[[2, 3254, 2], [2, 3254, 3], [2, 2460, 4]], [[2, 3277, 2], [2, 3277, 3], [2, 2483, 4]]]
len: 3: [[[3, 3328, 2], [3, 3328, 3], [3, 2534, 4]]]
tag 14 :
len: 1: [[[1, 2308, 1], [1, 3837, 10], [1, 3710, 2]]]
tag 15 :
len: 1: [[[2, 3689, 3], [2, 2895, 4], [2, 3394, 1]]]
tag 16 :


In [11]:
cleared_list

[[],
 [],
 [[[[4, 11, 4], [4, 11, 5], [4, 11, 6]]]],
 [],
 [],
 [[[[1, 858, 10], [1, 1607, 9], [1, 821, 4]]]],
 [[[[3, 928, 5], [3, 928, 6], [3, 1196, 1]]]],
 [],
 [[[[1, 2680, 9], [1, 2617, 2], [1, 2617, 3]]],
  [[[5, 2641, 9], [5, 2578, 2], [5, 2578, 3]]]],
 [[[[1, 2464, 7], [1, 2464, 8], [1, 1344, 5]],
   [[1, 2476, 7], [1, 2476, 8], [1, 1356, 5]]],
  [[[2, 2477, 7], [2, 2477, 8], [2, 1357, 5]]]],
 [],
 [],
 [],
 [[[[1, 3253, 2], [1, 3253, 3], [1, 2459, 4]]],
  [[[2, 3254, 2], [2, 3254, 3], [2, 2460, 4]],
   [[2, 3277, 2], [2, 3277, 3], [2, 2483, 4]]],
  [[[3, 3328, 2], [3, 3328, 3], [3, 2534, 4]]]],
 [[[[1, 2308, 1], [1, 3837, 10], [1, 3710, 2]]]],
 [[[[2, 3689, 3], [2, 2895, 4], [2, 3394, 1]]]],
 []]

In [13]:
2138 / 3

712.6666666666666

In [12]:
sum([0, 1680, 480, 2070, 1998, 378, 423, 2187, 2112, 462, 2138, 570, 1992, 540, 537, 462, 1995] )

20024